# LSTM을 사용하여 주가를 예측하자

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import FinanceDataReader as fdr

import tensorflow as tf
from keras.models import Sequential, load_model, model_from_json
from keras.layers import LSTM, Dense, Dropout
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.preprocessing import MinMaxScaler

In [2]:
###############################################################################
#                                                                             #
#                                  변수 설정                                  #
code = '036570'                                          # 종목코드           #
name = 'NCSOFT'                                        # 기업명             #
date_start = '2018-01-01'                                # 시작날짜           # 
date_end = '2021-08-31'                                  # 마지막날짜         #
look_back = 5                                            # Window크기         #
split_percent = 0.70                                     # 훈련데이터비율     #
batch_size = 32                                          # Batch 크기         #
periods = 30                                             # 예측기간           #
num_epochs = 100                                         # 모델학습 반복횟수  #
#                                                                             #
#                                                                             #
###############################################################################

In [3]:
# FinanceDataReader에서 주식정보 가져오기
stock = fdr.DataReader(code, date_start, date_end)
stock.describe()

Open          High           Low         Close        Volume  \
count  9.040000e+02  9.040000e+02  9.040000e+02  9.040000e+02  9.040000e+02   
mean   6.162992e+05  6.258811e+05  6.072633e+05  6.166521e+05  1.084684e+05   
std    1.968743e+05  2.000396e+05  1.939004e+05  1.966994e+05  7.649129e+04   
min    3.330000e+05  3.400000e+05  3.300000e+05  3.325000e+05  2.726100e+04   
25%    4.480000e+05  4.532500e+05  4.420000e+05  4.487500e+05  6.928375e+04   
50%    5.340000e+05  5.410000e+05  5.270000e+05  5.340000e+05  9.248050e+04   
75%    8.162500e+05  8.300000e+05  8.060000e+05  8.180000e+05  1.247482e+05   
max    1.028000e+06  1.048000e+06  1.002000e+06  1.038000e+06  1.048836e+06   

           Change  
count  904.000000  
mean     0.000692  
std      0.022907  
min     -0.152927  
25%     -0.012110  
50%      0.000000  
75%      0.011730  
max      0.134615

In [4]:
# DataFrame 안의 Value가 0인 부분 찾기(예측시 오차율 줄이기)
stock.loc[stock['Volume']==0]

Empty DataFrame
Columns: [Open, High, Low, Close, Volume, Change]
Index: []

In [5]:
# 먼저 0 을 NaN으로 바꾼후, Missing Data 처리
stock['Volume'] = stock['Volume'].replace(0, np.nan)
# Missing Data 처리
stock = stock.dropna()
stock.isnull().sum()

Open      0
High      0
Low       0
Close     0
Volume    0
Change    0
dtype: int64

In [6]:
# stock = stock.drop(columns=['Open', 'High', 'Low', 'Volume', 'Change'])
# stock.to_csv('StockCSV.csv')

In [7]:
# 예측대상 Data를 Plotly 그래프로 그리기
trace1 = go.Scatter(
    x = stock.index,
    y = stock['Close'],
    mode = 'lines',
    name = 'Data'
)
layout = go.Layout(
    title = "Stock",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace1], layout=layout)
fig.show()

In [8]:
# 정규화 (예측대상 Data 정규화)
scaler = MinMaxScaler()
scaled_stock = scaler.fit_transform(stock['Close'].values.reshape(-1,1))

In [9]:
# 학습데이터, 테스트데이터 생성
close_data = scaled_stock

split = int(split_percent*len(close_data))

close_train = close_data[:split]
close_test = close_data[split:]

date_train = stock.index[:split]
date_test = stock.index[split:]

print(len(close_train))
print(len(close_test))

632
272


In [10]:
# 학습, 테스트 시계열데이터 생성
train_generator = TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=batch_size)
test_generator = TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=1)

In [10]:
# 모델 구축 및 컴파일
model = Sequential()
model.add(LSTM(50, activation='elu', return_sequences = True, input_shape=(look_back,1)))
model.add(Dropout(0.2))
model.add(LSTM(50, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(1))
sgd = tf.keras.optimizers.SGD(learning_rate=0.01, decay=1e-7, momentum=0.9, nesterov=False)
model.compile(loss='mse', optimizer=sgd)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 5, 50)             10400     
_________________________________________________________________
dropout (Dropout)            (None, 5, 50)             0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 51        
Total params: 30,651
Trainable params: 30,651
Non-trainable params: 0
_________________________________________________________________


In [11]:
# 예측모델 저장 및 학습 조기종료 설정
h5 = 'Analysis/LSTM-({})-({})-({})-001.h5'.format(name, date_start, date_end)
checkpoint = ModelCheckpoint(h5,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             save_weights_only=True,
                             mode='auto')

early_stop = EarlyStopping(monitor='val_loss',
                           patience=10)

json = 'Analysis/LSTM-({})-({})-({})-001.json'.format(name, date_start, date_end)
model_json = model.to_json()
with open(json, 'w') as json_file:
    json_file.write(model_json)

In [11]:
# # 예측모델 불러오기
# def load_keras_model(json, h5, optimizer):
#     json_file = open(json, 'r')
#     loaded_model_json = json_file.read()
#     json_file.close()
#     model = model_from_json(loaded_model_json)
#     model.compile(optimizer=optimizer, loss='mse')
#     model.load_weights(h5)
#     return model
# model = load_keras_model('Analysis/LSTM-({})-({})-({})-001.json'.format(name, date_start, date_end),
#                          'Analysis/LSTM-({})-({})-({})-001.h5'.format(name, date_start, date_end),
#                          'adam')

In [13]:
# LSTM모델 학습!!!!!!(설정값에 따라 시간소요)
model.fit(train_generator, validation_data=test_generator, epochs=num_epochs,
          shuffle=False, callbacks=[checkpoint, early_stop])

Epoch 1/100
20/20 [==============================] - 6s 89ms/step - loss: 0.0195 - val_loss: 0.0773

Epoch 00001: val_loss improved from inf to 0.07730, saving model to Analysis\LSTM-(NCSOFT)-(2018-01-01)-(2021-08-31)-001.h5
Epoch 2/100
20/20 [==============================] - 1s 46ms/step - loss: 0.1012 - val_loss: 0.1992

Epoch 00002: val_loss did not improve from 0.07730
Epoch 3/100
20/20 [==============================] - 1s 44ms/step - loss: 0.0566 - val_loss: 0.1624

Epoch 00003: val_loss did not improve from 0.07730
Epoch 4/100
20/20 [==============================] - 1s 47ms/step - loss: 0.0445 - val_loss: 0.1292

Epoch 00004: val_loss did not improve from 0.07730
Epoch 5/100
20/20 [==============================] - 1s 42ms/step - loss: 0.0422 - val_loss: 0.1143

Epoch 00005: val_loss did not improve from 0.07730
Epoch 6/100
20/20 [==============================] - 1s 32ms/step - loss: 0.0343 - val_loss: 0.0915

Epoch 00006: val_loss did not improve from 0.07730
Epoch 7/100
20/

In [13]:
prediction = model.predict(test_generator)

close_data = close_data.reshape((-1))
close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
prediction = prediction.reshape((-1))

trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = date_test[look_back:],
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = stock.index[split:],
    y = close_data[split-1:],
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = "Stock Prediction",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()
# fig.write_json('stocktest2.json')

In [15]:
# 평균절대값백분율오차계산 (MAPE)
mape = np.sum(abs(close_test[look_back:]-prediction)/close_test[look_back:]) /len(close_test[look_back:]) 
print(mape)
accuracy = 100 - mape*100
print('정확도: ', round(accuracy,2), '%')

0.04471849873545789
정확도:  95.53 %


In [13]:
# 설정한 기간만큼 미래 예측하는 함수
def predict(num_prediction, model):
    prediction_list = close_data[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
    
    return prediction_list

def predict_dates(num_prediction):
    last_date = stock.index.values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

In [14]:
# 설정한 기간을 예측
forecast = predict(periods, model)
forecast_dates = predict_dates(periods)

In [15]:
# 다음날 주가 예측!!
close_next = scaler.inverse_transform(forecast.reshape(-1,1))
print('내일 주가: ', round(int(close_next[1,0]), -1), '원')

내일 주가:  713740 원


In [35]:
trace1 = go.Scatter(
    x = date_test,
    y = close_test,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = date_test[look_back:],
    y = prediction,
    mode='lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x=forecast_dates,
    y=forecast,
    mode = 'lines',
    name = 'Forecast'
)
layout = go.Layout(
    title = "Stock Forecasting",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()
# fig.write_json('stocktest.json')

In [32]:
# data_df = pd.DataFrame(close_test, index=[date_test], columns=['Close'])
# predict_df = pd.DataFrame(prediction, index=[date_test[look_back:]], columns=['Close'])
# forecast_df = pd.DataFrame(forecast, index=[forecast_dates], columns=['Close'])
# data_df.to_csv('StockCSV.csv')
# predict_df.to_csv('StockCSV.csv')
# forecast_df.to_csv('StockCSV.csv')

Close
2021-08-31  0.464210
2021-09-01  0.540385
2021-09-02  0.490580
2021-09-03  0.482317
2021-09-04  0.489324
2021-09-05  0.499197
2021-09-06  0.506007
2021-09-07  0.496591
2021-09-08  0.498642
2021-09-09  0.502613
2021-09-10  0.505298
2021-09-11  0.506193
2021-09-12  0.505953
2021-09-13  0.508100
2021-09-14  0.510069
2021-09-15  0.511487
2021-09-16  0.512649
2021-09-17  0.513932
2021-09-18  0.515578
2021-09-19  0.517060
2021-09-20  0.518428
2021-09-21  0.519800
2021-09-22  0.521228
2021-09-23  0.522685
2021-09-24  0.524090
2021-09-25  0.525481
2021-09-26  0.526880
2021-09-27  0.528286
2021-09-28  0.529684
2021-09-29  0.531069
2021-09-30  0.532451